# 1. Representação de textos

Nesse caderno vamos ver algumas técnicas de representação de features e de texto, como One-Hot, Bag of Words, Bag of N-grams e TF-IDF.

## 1.1 Codificação One-Hot

A codificação One-Hot consiste em representar cada possível palavra por um vetor binário de tamanho igual ao do corpus do texto analisado. Então, a posição em que tivermos o valor 1 indica qual a palavra.

In [ ]:
# Definindo sentenças com qual trabalharemos
textos = ["Cachorro morde homem.","Homem morde cachorro.","Cachorro come carne.","Homem come comida."]

In [ ]:
# Realizando um pequeno pré-processamento
textos_tratados = [texto.lower().replace(".","") for texto in textos]
textos_tratados

['cachorro morde homem',
 'homem morde cachorro',
 'cachorro come carne',
 'homem come comida']

In [ ]:
# Construção do vocabulário através de dicionário indexado
vocabulario = {}
cont = 0
for texto in textos_tratados:
    for palavra in texto.split():
        if palavra not in vocabulario:
            cont = cont + 1
            vocabulario[palavra] = cont
print(vocabulario)

{'cachorro': 1, 'morde': 2, 'homem': 3, 'come': 4, 'carne': 5, 'comida': 6}


In [ ]:
# Criando função que devolve one-hot encoding para esse vocabulário
# Caso a palavra não faça parte do vocabulário atribuimos a ela vetor 0
def criar_ohe(texto):
  codificado=[]
  for palavra in texto.split():
    aux = [0] * len(vocabulario)
    if palavra in vocabulario:
      aux[vocabulario[palavra]-1] = 1
    codificado.append(aux)
  return codificado

In [ ]:
# Testando função de ohe
print(textos_tratados)
criar_ohe(textos_tratados[0])

['cachorro morde homem', 'homem morde cachorro', 'cachorro come carne', 'homem come comida']


[[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0]]

## 1.2 Codificação One-Hot Encoding com scikit-learn

Agora, vamos usar a biblioteca scikit-learn para realizar a codificação OHE. 

In [ ]:
# Importando dependências
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
# Definindo textos e base de dados para criar vocabulário
textos = ["cachorro morde homem","homem morde cachorro","cachorro come carne","homem come comida"]
tokens = [texto.split() for texto in textos]
dados = []
for frase in tokens:
  for palavra in frase:
    dados.append(palavra)
dados

['cachorro',
 'morde',
 'homem',
 'homem',
 'morde',
 'cachorro',
 'cachorro',
 'come',
 'carne',
 'homem',
 'come',
 'comida']

In [ ]:
# Obtendo codificação para números inteiros com todos os elementos do vocabulário
encoder = LabelEncoder()
indices = encoder.fit_transform(dados)
print("Codificacao:",indices)

Codificacao: [0 5 4 4 5 0 0 2 1 4 2 3]


## 1.3 Representação bag of words

A representação bag of words representa as palavras do texto através de uma espécie de bolsa, ignorando contexto e ordem. 

Aqui também representamos palavras através de índices que vão de 0 até o tamanho do vocabulário, mas diferentes do OHE, a representação de uma frase é feita apenas com um vetor de |V| posições.

In [ ]:
# Sentenças que usaremos e pré-processamento
textos = ["Cachorro morde homem.","Homem morde cachorro.","Cachorro come carne.","Homem come comida."]
textos_tratados = [texto.lower().replace(".","") for texto in textos]
textos_tratados

['cachorro morde homem',
 'homem morde cachorro',
 'cachorro come carne',
 'homem come comida']

In [ ]:
# Importando dependências necessárias
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Criando vetorizador
vetorizador = CountVectorizer()

In [ ]:
# Criando representação de Bag of Words
bow = vetorizador.fit_transform(textos_tratados)

In [ ]:
# Mapeamento realizado pelo vetorizador
print("Vocabulario: ", vetorizador.vocabulary_)

Vocabulario:  {'cachorro': 0, 'morde': 5, 'homem': 4, 'come': 2, 'carne': 1, 'comida': 3}


In [ ]:
# Testando representação bow para os textos
print(textos_tratados[0])
print(bow[0].toarray())

cachorro morde homem
[[1 0 0 0 1 1]]


## 1.4 Bag of N-Grams

O conceito dessa representação nada mais é que uma generalização da bag of words.

In [ ]:
# Sentenças que usaremos e pré-processamento
textos = ["Cachorro morde homem.","Homem morde cachorro.","Cachorro come carne.","Homem come comida."]
textos_tratados = [texto.lower().replace(".","") for texto in textos]
textos_tratados

['cachorro morde homem',
 'homem morde cachorro',
 'cachorro come carne',
 'homem come comida']

In [ ]:
# Aqui também vamos usar o vetorizador, mas passaremos um parâmetro adicional que indicará que queremos n-gramas de i até j
from sklearn.feature_extraction.text import CountVectorizer
vetorizador = CountVectorizer(ngram_range=(1,3))
bon_rep = vetorizador.fit_transform(textos_tratados)
print("Vocabulario: ", vetorizador.vocabulary_)

Vocabulario:  {'cachorro': 0, 'morde': 15, 'homem': 10, 'cachorro morde': 3, 'morde homem': 17, 'cachorro morde homem': 4, 'homem morde': 13, 'morde cachorro': 16, 'homem morde cachorro': 14, 'come': 6, 'carne': 5, 'cachorro come': 1, 'come carne': 7, 'cachorro come carne': 2, 'comida': 9, 'homem come': 11, 'come comida': 8, 'homem come comida': 12}


In [ ]:
# Agora, vamos ver representações das nossas sentenças através da bag
bon_rep[0].toarray()

array([[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1]])

## 1.5 TF-IDF (Term Frequency-Inverse Document Frequency)

Nas outras abordagens de representação que trabalhamos todas as palavras tinham a mesma importância. Na abordagem do TF-IDF (Frequência do Termo - Inverso da Frequência no Documento) ocorre uma quantificação da importância de uma palavra. Normalmente, é um sistema usado na recuperação de informações.

Para calcular a importância usamos o score que é a subtração entre a frequência da palavra num documento di (TF) e o inverso da frequência da palavra no corpus de todos os documentos dj (IDF).

A representação vetorial de TF-IDF é então, simplesmente, a pontuação TF-IDF para cada termo nesse documento.

In [ ]:
# Sentenças que usaremos e pré-processamento
textos = ["Cachorro morde homem.","Homem morde cachorro.","Cachorro come carne.","Homem come comida."]
textos_tratados = [texto.lower().replace(".","") for texto in textos]
textos_tratados

['cachorro morde homem',
 'homem morde cachorro',
 'cachorro come carne',
 'homem come comida']

In [ ]:
# Importando TF-IDF do scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Criando objeto codificador sobre os textos
tfidf = TfidfVectorizer()
tfidf_rep = tfidf.fit_transform(textos_tratados)

In [ ]:
# Todas as palavras no vocabulário
print("Palavras: ",tfidf.get_feature_names_out())

Palavras:  ['cachorro' 'carne' 'come' 'comida' 'homem' 'morde']


In [ ]:
# Mostrando IDF de cada palavra
print("IDFs: ",tfidf.idf_)

IDFs:  [1.22314355 1.91629073 1.51082562 1.91629073 1.22314355 1.51082562]


In [ ]:
# Mostrando a representação TF-IDF de cda documento do corpus
print("Representação TFIDF", tfidf_rep.toarray())

Representação TFIDF [[0.53256952 0.         0.         0.         0.53256952 0.65782931]
 [0.53256952 0.         0.         0.         0.53256952 0.65782931]
 [0.44809973 0.70203482 0.55349232 0.         0.         0.        ]
 [0.         0.         0.55349232 0.70203482 0.44809973 0.        ]]


# 2. Convertendo textos para features

Vamos usar as técnicas de representação de texto que estudamos para criar features.

In [ ]:
# Importando pandas
import pandas as pd

## 2.1 One-Hot Encoding

In [ ]:
# Definindo sentença no qual trabalharemos
sentenca = "São Paulo é o maior do Brasil"

In [ ]:
# Criando tabela pandas com atributos ohe
pd.get_dummies(sentenca.split())

,Brasil,Paulo,São,do,maior,o,é
0,0,0,1,0,0,0,0
1,0,1,0,0,0,0,0
2,0,0,0,0,0,0,1
3,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0
5,0,0,0,1,0,0,0
6,1,0,0,0,0,0,0


## 2.2 Count Vectorizer

Diferente da abordagem anterior nessa representação também consideramos a frequência das palavras na representação.

In [ ]:
# Importando dependência
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Definindo texto usado para teste
texto = ["Time grande cai, mas time gigante não cai"]

In [ ]:
# Criando objeto de vetorização e realizando o fit
vetorizador = CountVectorizer()
vetorizador.fit(texto)

CountVectorizer()

In [ ]:
# Olhando o vocabulário do vetorizador
print(vetorizador.vocabulary_)

{'time': 5, 'grande': 2, 'cai': 0, 'mas': 3, 'gigante': 1, 'não': 4}


In [ ]:
# Olhando representação da nossa sentença com codificação vetorizada
vetor = vetorizador.transform(texto)
print(vetor.toarray())

[[2 1 1 1 1 2]]


## 2.3 N-Gramas

In [ ]:
# Importando vetorizador com contagem
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Texto que usaremos
texto = ["Eu amo PLN e eu irei aprender PLN em 3 meses"]

In [ ]:
# Criando a vetorização através de 2-gramas
vetorizador = CountVectorizer(ngram_range=(2, 2))

In [ ]:
# Treinando o vetorizador e vendo vocabulário
vetorizador.fit(texto)
print(vetorizador.vocabulary_)

{'eu amo': 3, 'amo pln': 0, 'pln eu': 7, 'eu irei': 4, 'irei aprender': 5, 'aprender pln': 1, 'pln em': 6, 'em meses': 2}


In [ ]:
# Codificando nosso texto e mostrando resultado
vetor = vetorizador.transform(texto)
print(vetor.toarray())

[[1 1 1 1 1 1 1 1]]


## 2.4 TF-IDF

In [ ]:
# Importando módulo
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Definindo os textos
textos = ["A ligeira raposa marrom saltou sobre o cachorro preguiçoso.", "O cachorro.", "A raposa"]

In [ ]:
# Criando o objeto transformador
vetorizador = TfidfVectorizer()

In [ ]:
# Treinando o transformador
vetorizador.fit(textos)

TfidfVectorizer()

In [ ]:
# Mostrando vocabulário
print(vetorizador.vocabulary_)

{'cachorro': 0, 'morde': 5, 'homem': 4, 'come': 2, 'carne': 1, 'comida': 3}


In [ ]:
# Mostrando representação TF-IDF
print(vetorizador.idf_)

[1.22314355 1.91629073 1.51082562 1.91629073 1.22314355 1.51082562]


Parei no notebook 11